In [1]:
from pyquil import Program
from pyquil.gates import *
from pyquil.quil import address_qubits
from pyquil.quilatom import QubitPlaceholder
from pyquil.api import QVMConnection
from pyquil.api import WavefunctionSimulator

import networkx as nx 
from toric_code import * 

# Toric Code Tutorial

Hello there! This notebook explains the basic concepts of our implementation of the toric code, which is described in detail in our final paper. We'll discuss how to use the code, and also the major problems we've run into in determining the error threshold. This notebook assumes that you've installed the latest release of ``PyQuil``, ``Numpy``, and ``NetworkX``. 

To begin, let's construct the primal, dual, and distance graphs for a specified lattice size $L$. We'll pick $L = 3$ for simplicity here. 

In [10]:
L = 3
primal, dual, distance = construct_toric_code(L)

The function ``construct_toric_code`` returns ``NetworkX`` graph structures. For details, see the ``NetworkX`` documentation, but note that we can access the edges and nodes of the primal and dual graphs as necessary. Each edge contains a ``QubitPlaceholder`` object representing a data qubit, while each vertex contains a ``QubitPlaceholder`` object representing the $X$ stabilizers (on the primal graph) or $Z$ stabilizers (on the dual graph). 

The next step in our protocl is to initialize the qubits to $+1$ eigenstates of the $X$ stabilizers and $Z$ stabilizers. Unfortunately, this has proven to be the major limiting factor in our experiment, as constructing this initial state is highly non-trivial. We've attempted to follow the method described in Dennis et al. (http://info.phys.unm.edu/~alandahl/papers/jmp43_4452.pdf?fbclid=IwAR3XnFXFsdVkoUs9iCp0oA-ssHtflBFpPubNwNZxQ_BNvmjcCldh0xhd7mg), which involves the following steps: 

1) Initialize the qubits to the state $|0\rangle^{\otimes n}$, where $n$ is the number of physical qubits. 

2) Without applying any errors, run the $X$ syndrome extraction protocol, which involves the circuit shown in Figure 1 of our final paper. 

3) After the $X$ syndrome extraction, about half of our ancilla qubits are expected to be in the $|1\rangle$ state, which corresponds to a measured error (that is, a $-1$ eigenvalue upon measuring the $X$ stabilizer). Construct an arbitrary one-chain with these error nodes on the boundaries (in this case, a "one-chain" is simply a collection of edges between vertices in the graph; note that these edges do not have to form one continuous path). Then apply a $Z$ operator to the data qubit on each edge of this one-chain. 

After this procedure, the data qubits of the toric code are in a $+1$ eigenstate of the stabilizer operators. We started an implementation of this initialization procedure in the function ``initialize_toric_code`` below. Unfortunately, we haven't yet figured out a good algorithm for generating a one-chain that has error nodes as a boundary. We're currently working on this problem, but it's been a lot more difficult than we were expecting! 

In [6]:
def initialize_toric_code(primal, dual, L): 
    primal_initial = Program() 
    dual_initial = Program() 
    primal_faulty_nodes = syndrome_extraction(primal, L, primal_initial, "X")
    dual_faulty_nodes = syndrome_extraction(dual, L, dual_initial, "Z")
    print(primal_faulty_nodes)
    # syndrome_extraction returns a list of the nodes with errors 
    # At this point, we construct a one-chain with these error nodes as 
    # a boundary; to do this, we'll find the shortest path between 
    # each pair of nodes in the faulty_nodes lists ==> Will this leave nodes on boundary? 
    return primal_initial

primal_test = initialize_toric_code(primal, dual, L)

[(0, 0), (1, 1), (2, 0), (2, 2)]


The function ``syndrome_extraction`` above is one of the most important functions of our code. Given a graph (primal or dual) and a stabilizer operator, it measures the stabilizer operator and returns a list of nodes (ancilla qubits) that returned the $|1\rangle$ state (a $-1$ eigenvalue, indicating an error). 

After initializing the toric code, we can begin a simulation of the toric code, which is given in our ``main`` function. The first step of the simulation is to apply errors to the code under the independent noise assumption (see our paper for further details). 

In [10]:
# Assuming the data qubits are correctly initialized, we need to simulate an error 
# Working under the independent noise assumption; note that we must set a p value 
primal_pq, phase_flips, dual_pq, bit_flips = simulate_error(primal, dual, p=0.2)
print(primal_pq) # Operators for random phase flips
print(dual_pq) # Operators for random bit flips 

Z {q43451957320}
Z {q43452624968}
Z {q43452625248}

X {q43452625080}
X {q43451957544}
X {q43451957600}



The function ``simulate_error`` returns programs for the primal and dual graphs representing the bit/phase flip operations, as well as lists of edges representing where bit/phase flips were applied. Now, we need to run the ``syndrome_extraction`` algorithm for each graph to determine where bit and phase flips occured. Note that this is the same function used in ``initialize_toric_code`` above. 

In [12]:
faulty_primal_nodes = syndrome_extraction(primal, L, primal_pq, "X") # Extraction for X stabilizers on primal graph 
faulty_dual_nodes = syndrome_extraction(dual, L, dual_pq, "Z") # Extraction for Z stabilizers on dual graph 
print(faulty_primal_nodes)
print(faulty_dual_nodes)

[(0, 0), (1, 1), (1, 2), (2, 1), (2, 2)]
[(0, 1), (1, 0), (1, 1), (1, 2), (2, 1), (2, 2)]


Great! Now that we've identified where the errors are, we need to correct them. For this, we use the minimum-weight perfect-matching algorithm (MWPM), as described in our final report. This algorithm is implemented (using ``NetworkX``, with a few modifications) in our code as ``mwpm``. 

In [13]:
primal_correction_paths = mwpm(primal, distance, L, faulty_primal_nodes) # Corrections for primal graph 
dual_correction_paths = mwpm(dual, distance, L, faulty_dual_nodes) # Corrections for dual graph 

AssertionError: 

NOTE an important point here: a perfect matching is not always possible, given the errors we applied! In particular, if an odd number of errors occur, we cannot generate a perfect matching to identify a correction path for the code. In this case, we're raising an ``AssertionError`` because a perfect matching is not possible, which is expected because our qubits were not initialized correctly. 

To correct for the indicated errors, we use the ``apply_operation`` function. Given a correction path and correction operator, this function walks along the path and applies the correction operator to each data qubit. 

In [ ]:
primal_correct_pq = apply_operation(primal_correction_paths, primal, Z)
dual_correct_pq = apply_operation(dual_correction_paths, dual, X)
# We won't run this cell, since the mwpm algorithm failed and thus the correction paths 
# do not exist 

In [ ]:
# Now, we just run the correction programs 
qvm = QVMConnection() 
qvm.run(primal_correct_pq)
qvm.run(dual_correct_pq)
# Again, we won't run this cell becuase of the mwpm algorithm 

This completes one complete cycle of our simulation. To compute the error threshold, we simply repeat this process while varying $L$ and $p$, and measure the average quantum state lifetime as a function of these parameters. Due to our issues with initialization, we unfortunately haven't been able to get this computation to work, but we plan to implement it once we solve our initialization problem (this unfortuntately might occur too late for it to impact our grade at all... but alas, such is the risk of pursuing the toric code in a final project!). 